<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/intel_images_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 16th October, 2021
# Ref: https://www.kaggle.com/puneet6060/intel-image-classification
#     https://www.kaggle.com/accountstatus/using-keras-cnn-on-intel-data

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten,Conv2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image
import matplotlib.pyplot as plt
import os


In [ ]:
# Creating the model

model=Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(300,activation='relu'))
model.add(Dense(units=6,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
                           model,
                           to_file="model.png",
                           show_shapes=False,
                           show_layer_names=True,
                           rankdir="TB",
                           expand_nested=False
                          )

In [ ]:
model.compile(
              optimizer= 'adam',
              loss= 'categorical_crossentropy',
              metrics= ['accuracy']
              )

In [ ]:
train_datagen = ImageDataGenerator(
                                   rescale = 1./255, 
                                   #shear_range = 0.2, 
                                   #zoom_range = 0.2, 
                                   horizontal_flip = True
                                   )

In [ ]:
# Mount gdrive
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
train_path = '/gdrive/MyDrive/Colab_data_files/intel_images/seg_train' 
training_set = train_datagen.flow_from_directory(
                                                  train_path,
                                                  target_size = (64, 64), 
                                                  batch_size = 32
                                                 )


In [ ]:
test_path = '/gdrive/MyDrive/Colab_data_files/intel_images/seg_test'
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
                                            test_path,
                                            target_size = (64, 64),
                                            batch_size = 32
                                            )

In [ ]:
len(set(training_set.classes))

In [ ]:
fitted_model = model.fit_generator(
                                    training_set,
                                    steps_per_epoch = 50,
                                    epochs = 5,
                                    validation_data = test_set,
                                    validation_steps = 10
                                   )

In [ ]:
! pip install pyyaml h5py

In [ ]:
from tensorflow.keras.models import save_model

In [ ]:
! mkdir /gdrive/MyDrive/Colab_data_files/intelmodel

In [ ]:
pathToSave = '/gdrive/MyDrive/Colab_data_files/intelmodel'
model.save(pathToSave)

In [ ]:
mymodel = keras.models.load_model(pathToSave)

In [ ]:
image_path_forest = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/forest/10007.jpg"
image_path_building = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/buildings/10006.jpg"
image_path_glacier = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/glacier/10.jpg"
image_path_mountain = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/mountain/10008.jpg"
image_path_sea = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/sea/10016.jpg"
image_path_street = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/street/10015.jpg"

In [ ]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import img_to_array
def predict(image_path): 
    model = keras.models.load_model(pathToSave)
    image = load_img(image_path, target_size=(64, 64))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    #image = preprocess_input(image)
    yhat = model.predict(image)
    classes_x=np.argmax(yhat,axis=1)
    l=os.listdir(train_path)
    l.sort()
    return l[classes_x[0]]

In [ ]:
predict(image_path_street)

In [ ]:
plt.plot([i for i in range(20)],fitted_model.history['accuracy'])
plt.plot([i for i in range(20)],fitted_model.history['loss'])

In [ ]:
def image_prediction(path):
    imm=Image.open(path)
    imm=imm.resize((64,64))
    x=np.array(imm)
    x=np.expand_dims(x,axis=0)
    classs=model.predict_classes(x)
    l=os.listdir('../input/intel-image-classification/seg_train/seg_train')
    l.sort()
    return l[classs[0]]